In [4]:
import pandas as pd
import os
import glob
import requests
import sys



In [ ]:
# connect to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
# define input and output path
input_path = input('Enter inputfile path: ')
# Create an output file path based on the input
dir_path = os.path.dirname(input_path)
output_path = os.path.join(dir_path, 'Output_annotation.tsv')
print(input_path)
print(output_path)


Enter inputfile path: /content/drive/My Drive/Colab Notebooks/test_vcf_data.txt
/content/drive/My Drive/Colab Notebooks/test_vcf_data.txt
/content/drive/My Drive/Colab Notebooks/Output_annotation.tsv


In [ ]:
# Read the VCF file into pandas dataFrame, using comment to exclude information lines from the vcf file

vcf_df = pd.read_csv(input_path,comment = '#' ,sep ='\t', header=None)



# Include names as listed in VCF file
vcf_df.columns =['CHROM',	'POS',	'ID',	'REF'	,'ALT',	'QUAL'	,'FILTER'	,'INFO'	,'FORMAT'	,'sample']
# Print the first few rows of the DataFrame
print(vcf_df.head())





  CHROM      POS ID REF ALT  QUAL FILTER  \
0     1  1158631  .   A   G  2965   PASS   
1     1  1246004  .   A   G  2965   PASS   
2     1  1249187  .   G   A  2965   PASS   
3     1  1261824  .   G   C  2965   PASS   
4     1  1387667  .   C   G  2965   PASS   

                                                INFO              FORMAT  \
0  BRF=0.16;FR=1.0000;HP=1;HapScore=1;MGOF=3;MMLQ...  GT:GL:GOF:GQ:NR:NV   
1  BRF=0.09;FR=1.0000;HP=6;HapScore=1;MGOF=5;MMLQ...  GT:GL:GOF:GQ:NR:NV   
2  BRF=0.16;FR=1.0000;HP=3;HapScore=1;MGOF=3;MMLQ...  GT:GL:GOF:GQ:NR:NV   
3  BRF=0.15;FR=1.0000;HP=1;HapScore=1;MGOF=5;MMLQ...  GT:GL:GOF:GQ:NR:NV   
4  BRF=0.17;FR=1.0000;HP=2;HapScore=1;MGOF=3;MMLQ...  GT:GL:GOF:GQ:NR:NV   

                               sample  
0  1/1:-300.0,-43.88,0.0:3:99:160:156  
1  1/1:-300.0,-41.24,0.0:5:99:152:148  
2  1/1:-300.0,-37.63,0.0:3:99:137:135  
3  1/1:-300.0,-39.74,0.0:5:99:136:134  
4  1/1:-300.0,-39.41,0.0:3:99:137:133  


In [ ]:
# Extract Depth of sequence coverage at the site of variation from INFO, as per the definition provided in the vcf file
vcf_df['DP'] = vcf_df['INFO'].str.extract(r'QD=(\d+)').astype(float)


In [ ]:
# Extract Number of reads supporting the variant from INFO, as per the definition provided in the vcf file
vcf_df['Total Reads'] = vcf_df['INFO'].str.extract(r'TR=(\d+)').astype(float)

In [ ]:
# This is an additional column extracted to compute the perecntage in later step.
vcf_df['Forward read'] = vcf_df['INFO'].str.extract(r'NF=(\d+)').astype(float)

In [ ]:
# compute Percentage of reads supporting the variant versus those supporting reference reads, as per the definition provided in the vcf file
vcf_df['percentage supporting reads'] =round((((( vcf_df['Total Reads']- vcf_df["Forward read"]))/vcf_df['Total Reads']) * 100),2)

In [ ]:
vcf_df.head()

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,sample,DP
0,1,1158631,.,A,G,2965,PASS,BRF=0.16;FR=1.0000;HP=1;HapScore=1;MGOF=3;MMLQ...,GT:GL:GOF:GQ:NR:NV,"1/1:-300.0,-43.88,0.0:3:99:160:156",NaN
1,1,1246004,.,A,G,2965,PASS,BRF=0.09;FR=1.0000;HP=6;HapScore=1;MGOF=5;MMLQ...,GT:GL:GOF:GQ:NR:NV,"1/1:-300.0,-41.24,0.0:5:99:152:148",NaN
2,1,1249187,.,G,A,2965,PASS,BRF=0.16;FR=1.0000;HP=3;HapScore=1;MGOF=3;MMLQ...,GT:GL:GOF:GQ:NR:NV,"1/1:-300.0,-37.63,0.0:3:99:137:135",NaN
3,1,1261824,.,G,C,2965,PASS,BRF=0.15;FR=1.0000;HP=1;HapScore=1;MGOF=5;MMLQ...,GT:GL:GOF:GQ:NR:NV,"1/1:-300.0,-39.74,0.0:5:99:136:134",NaN
4,1,1387667,.,C,G,2965,PASS,BRF=0.17;FR=1.0000;HP=2;HapScore=1;MGOF=3;MMLQ...,GT:GL:GOF:GQ:NR:NV,"1/1:-300.0,-39.41,0.0:3:99:137:133",NaN


In [ ]:

# Function to fetch variant information
def fetch_variant_info(row):
     # define the server and hgvs notations for API call
    server = "https://grch37.rest.ensembl.org"
    hgvs_notation = f"{row['CHROM']}:g.{row['POS']}{row['REF']}>{row['ALT']}"
    ext = "/vep/human/hgvs/"+hgvs_notation
    # adding parameters to get back variant class and all other required annotation information
    params = {
        "species": "homo_sapiens",
        "assembly": "GRCh37",
        "variant_set": "default",
        "format": "json",
        "variant_class": "1",
        "hgvs": "1",
        "content-type": "application/json",
    }

   # below is get api calls and parse the json ouput to get appropriate required annotation and if there are any error feteching than return N/A
    try:
        response = requests.get(server+ext, headers={ "Content-Type" : "application/json"}, json=params)
        response.raise_for_status()
        decoded = response.json()[0]  # Assuming only one variant per request

        variant_effect = decoded.get("most_severe_consequence", "N/A")
        gene = decoded.get("transcript_consequences", [{}])[0].get("gene_symbol", "N/A")
        gene_id = decoded.get("transcript_consequences", [{}])[0].get("gene_id", "N/A")
        variant_impact = decoded.get("transcript_consequences", [{}])[0].get("impact", "N/A")
        variant_consequence = decoded.get("transcript_consequences", [{}])[0].get("consequence_terms", "N/A")
        variant_class = decoded.get("variant_class", "N/A")
        minor_allele_freq = decoded.get("colocated_variants", [{}])[0].get("frequencies", {}).get({row['ALT']}, {}).get("af", "N/A")
        clinical_significance = decoded.get("colocated_variants", [{}])[0].get("clin_sig", "N/A")
        clinical_significance_allele = decoded.get("colocated_variants", [{}])[0].get("clin_sig_allele", "N/A")
        phenotype = decoded.get("colocated_variants", [{}])[0].get("phenotype_or_disease", "N/A")
# here inluded additional annotation like clinical significance and clinical allele as well as phenotype to get more insigits of the variant allele and associate with any known disease before
        return variant_effect, gene, gene_id, variant_impact, variant_consequence, variant_class, minor_allele_freq, clinical_significance, clinical_significance_allele, phenotype

    except requests.RequestException as e:
        print(f"Error fetching data for {hgvs_notation}: {e}")
        return "N/A", "N/A", "N/A", "N/A", "N/A", "N/A", "N/A", "N/A", "N/A", "N/A"

# Apply the function to each row in the DataFrame
vcf_df[["variant_effect", "gene", "gene_id", "variant_impact", "variant_consequence",
              "variant_class", "minor_allele_freq", "clinical_significance",
              "clinical_significance_allele", "phenotype"]] = vcf_df.apply(fetch_variant_info, axis=1, result_type="expand")

# Now variants_df contains the additional columns
final_columns = ["CHROM", "POS", "REF", "ALT","DP","Total Reads","Forward read", "percentage supporting reads",
                 "gene", "gene_id", "variant_effect", "variant_consequence",
                 "variant_class","variant_impact", "minor_allele_freq", "clinical_significance",
                 "clinical_significance_allele", "phenotype"]
final_df = vcf_df[final_columns]

#print(vcf_df)


Error fetching data for 1:g.1647722GCTGTGACA>TCTAGGATG: 400 Client Error: Bad Request for url: https://grch37.rest.ensembl.org/vep/human/hgvs/1:g.1647722GCTGTGACA%3ETCTAGGATG
Error fetching data for 1:g.1647745GGCCCTTTC>AGCCCTTTT: 400 Client Error: Bad Request for url: https://grch37.rest.ensembl.org/vep/human/hgvs/1:g.1647745GGCCCTTTC%3EAGCCCTTTT
Error fetching data for 1:g.1647983TGGCTTAC>AGGCTTAT: 400 Client Error: Bad Request for url: https://grch37.rest.ensembl.org/vep/human/hgvs/1:g.1647983TGGCTTAC%3EAGGCTTAT
Error fetching data for 1:g.1650787TGATGCCTACATTTT>CGATGCCTACGTTTC: 400 Client Error: Bad Request for url: https://grch37.rest.ensembl.org/vep/human/hgvs/1:g.1650787TGATGCCTACATTTT%3ECGATGCCTACGTTTC
Error fetching data for 1:g.6219287TCACACA>TCACA,T: 400 Client Error: Bad Request for url: https://grch37.rest.ensembl.org/vep/human/hgvs/1:g.6219287TCACACA%3ETCACA,T
Error fetching data for 1:g.12921332TG>CA: 400 Client Error: Bad Request for url: https://grch37.rest.ensembl.org

In [ ]:
final_df.tail()

,CHROM,POS,REF,ALT,DP,Total Reads,Forward read,percentage supporting reads,gene,gene_id,variant_effect,variant_consequence,variant_class,variant_impact,minor_allele_freq,clinical_significance,clinical_significance_allele,phenotype
11760,X,154020114,C,A,20.0,216.0,78.0,63.89,MPP1,ENSG00000130830,synonymous_variant,[synonymous_variant],N/A,LOW,N/A,N/A,N/A,N/A
11761,X,154456747,A,G,20.0,224.0,88.0,60.71,VBP1,ENSG00000155959,missense_variant,[missense_variant],N/A,MODERATE,N/A,N/A,N/A,1
11762,X,155125435,AG,A,20.0,178.0,3.0,98.31,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
11763,X,155127675,A,G,20.0,134.0,132.0,1.49,VAMP7,ENSG00000124333,intron_variant,[intron_variant],N/A,MODIFIER,N/A,N/A,N/A,N/A
11764,X,155233098,T,C,20.0,116.0,66.0,43.10,IL9R,ENSG00000124334,splice_polypyrimidine_tract_variant,"[splice_polypyrimidine_tract_variant, intron_v...",N/A,LOW,N/A,N/A,N/A,1


In [ ]:
# saving annotation ouput as tsv file
final_df.to_csv(output_path, sep="\t")